Specify eval folder and method list

In [1]:
folder_name_maybritt = 'full2'
folder_name_benchmarks = "full"
# method_list = ['nn_det_per_variable', 'idr', 'analogues', 'gan', 'corrdiff_fixed-seeds', 'corrdiff', "maybritt_nicolai_zeros-constant_dec-1e-3_onehot", "maybritt_nicolai_zeros-constant_dec-1e-3_onehot_temporal"]
method_list = ['nn_det_per_variable', 'idr', 'analogues', 'gan', 'corrdiff_fixed-seeds', "maybritt_nicolai_zeros-constant_dec-1e-3_onehot", "maybritt_nicolai_zeros-constant_dec-1e-3_onehot_temporal"]
method_name_dict = {
        "maybritt_nicolai_zeros-constant_dec-1e-3_onehot": 'EnScale',
        "maybritt_nicolai_zeros-constant_dec-1e-3_onehot_temporal": 'EnScale-t',
        "nn_det_per_variable": "NN-det",
        "gan": "GAN",
        "analogues": "Analogues",
        "idr": "EasyUQ",
        "corrdiff": "CorrDiff-s",
        "corrdiff_fixed-seeds": "CorrDiff"
    }
version_name = "vnewlayers2"
maybritt_runid = "maybritt_nicolai_zeros-constant_dec-1e-3_onehot"

## General ES and CRPS table

### Interpolation

In [2]:
import pandas as pd
import numpy as np 

In [3]:
latex_tables = []
    
for var in ['tas', 'pr', 'sfcWind', 'rsds']:
    if folder_name_maybritt is not None:
        df = pd.read_csv(f"output_evals/maybritt/interpolation/{folder_name_maybritt}/{var}/concatenated_results_2030-2039_multiple-methods_2378111217.csv")
    if folder_name_benchmarks is not None:
        df_bench = pd.read_csv(f"output_evals/benchmarks/interpolation/{folder_name_benchmarks}/{var}/concatenated_results_2030-2039_maybritt_3step_dense-conv_v3_2378111217.csv")
    
    if folder_name_maybritt is not None and folder_name_benchmarks is not None:
        df = pd.concat([df, df_bench])
    elif folder_name_benchmarks is not None:
        df = df_bench
    
    df = df.set_index('method').loc[method_list].reset_index()

    col_list = []
    for name in ["MSE", "ES_pool-None_kernel-1", "Score1_pool-None_kernel-1", "Score2_pool-None_kernel-1", "CRPS_pool-None_kernel-1", "CRPS_pool-max_kernel-10", "LSD-daily"]:
        col_list.append("mean*" + name)
        col_list.append("se*" + name)

    df_sel = df[["method"] + col_list] 
    for col in df_sel.columns:
        if col.startswith('mean*'):
            se_col = col.replace('mean*', 'se*')
            #df_sel[col] = df_sel[col].round(1)
            df_sel[se_col] = df_sel[se_col].round(2)
            
            if not (col in ["mean*Score1_pool-None_kernel-1", "mean*Score2_pool-None_kernel-1"]):
                min_idx = df_sel[col].idxmin()
                second_min_idx = df_sel[col].nsmallest(2).idxmax()
                if col in ["mean*CRPS_pool-None_kernel-1", "mean*CRPS_pool-max_kernel-10"]:
                    df_sel[col] = df_sel[col].apply(lambda x: f'\\textbf{{{x:.2f}}}' if x == df_sel[col].iloc[min_idx] else (f'\\underline{{{x:.2f}}}' if x == df_sel[col].iloc[second_min_idx] else f'{x:.2f}'))
                else:
                    # df_sel[col] = df_sel[col].apply(lambda x: f'\\textbf{{{x:.1f}}}' if x == df_sel[col].iloc[min_idx] else f'{x:.1f}')
                    df_sel[col] = df_sel[col].apply(lambda x: f'\\textbf{{{x:.1f}}}' if x == df_sel[col].iloc[min_idx] else (f'\\underline{{{x:.1f}}}' if x == df_sel[col].iloc[second_min_idx] else f'{x:.1f}'))
            else:
                df_sel[col] = df_sel[col].apply(lambda x: f'{x:.1f}')
                
            
            if not (col in ["mean*Score1_pool-None_kernel-1", "mean*Score2_pool-None_kernel-1"]):
                df_sel[col] = df_sel[col].astype(str) + ' ± ' + "\\footnotesize{" + df_sel[se_col].astype(str) + "}"
            df_sel.rename(columns={col: col.replace('mean*', '')}, inplace=True)
            df_sel = df_sel.drop(columns=[se_col])

        #elif col == "LSD":
        #    min_idx = df_sel[col].idxmin()
        #    second_min_idx = df_sel[col].nsmallest(2).idxmax()
        #    # df_sel[col] = df_sel[col].apply(lambda x: f'\\textbf{{{x:.3f}}}' if x == df_sel[col].iloc[min_idx] else f'{x:.3f}')
        #    df_sel[col] = df_sel[col].apply(lambda x: f'\\textbf{{{x:.3f}}}' if x == df_sel[col].iloc[min_idx] else (f'\\underline{{{x:.3f}}}' if x == df_sel[col].iloc[second_min_idx] else f'{x:.3f}'))

    df_sel.rename(columns={"MSE": "MSE $\\downarrow$",
                            "ES_pool-None_kernel-1": "ES $\\downarrow$", 
                           "Score1_pool-None_kernel-1": "ES$_\\text{pred}$", "Score2_pool-None_kernel-1": "ES$_\\text{var}$", 
                           "CRPS_pool-None_kernel-1": "CRPS $\\downarrow$", "CRPS_pool-max_kernel-10": "CRPS-mp $\\downarrow$",
                           "LSD-daily": "RALSD $\\downarrow$"}, inplace=True)

    #df_sel = pd.concat([df_sel, df[["CRPS-pixelwise", "CRPS-max-pool-10x10"]]], axis=1) # TO DO after standard errors
    #df_sel.rename(columns={"CRPS-pixelwise": "CRPS-pw", "CRPS-max-pool-10x10": "CRPS-max-pool-10"}, inplace=True)

    # Print the LaTeX table
    # Remove trailing zeros
    # df_sel[df_sel.select_dtypes(include=[np.number]).columns] = df_sel.select_dtypes(include=[np.number]).applymap(lambda x: '{:.1f}'.format(x).rstrip('0').rstrip('.') if pd.notnull(x) else '')
        

    df_sel['method'] = df_sel['method'].replace(method_name_dict)
    # Convert the DataFrame to a LaTeX table
    latex_table = df_sel.to_latex(index=False, na_rep='')
    
    latex_tables.append(latex_table)

# Concatenate LaTeX tables with \multicolumn and \midrule between variables
final_latex_table = ""
final_latex_table += "\\begin{tabular}{llllllll}\n"
for var, table in zip(['tas', 'pr', 'sfcWind', 'rsds'], latex_tables):
    final_latex_table += f"\\midrule\n\\multicolumn{{7}}{{c}}{{{var}}} \\\\\n\\midrule\n"
    final_latex_table += table.split("\\toprule")[1].split("\\bottomrule")[0].strip() + "\n"
final_latex_table += "\\end{tabular}"
print(final_latex_table)
# Save the final LaTeX table to a .tex file
with open(f'tables/table_energy_scores_crps_lsd_{version_name}.tex', 'w') as f:
        f.write(final_latex_table)

/tmp/ipykernel_1642209/2588688811.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel[se_col] = df_sel[se_col].round(2)
/tmp/ipykernel_1642209/2588688811.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel[col] = df_sel[col].apply(lambda x: f'\\textbf{{{x:.1f}}}' if x == df_sel[col].iloc[min_idx] else (f'\\underline{{{x:.1f}}}' if x == df_sel[col].iloc[second_min_idx] else f'{x:.1f}'))
/tmp/ipykernel_1642209/2588688811.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

\begin{tabular}{llllllll}
\midrule
\multicolumn{7}{c}{tas} \\
\midrule
method & MSE $\downarrow$ & ES $\downarrow$ & ES$_\text{pred}$ & ES$_\text{var}$ & CRPS $\downarrow$ & CRPS-mp $\downarrow$ & RALSD $\downarrow$ \\
\midrule
NN-det & \textbf{4.5} ± \footnotesize{0.08} & 245.5 ± \footnotesize{1.8} & 245.4 & 0.0 & 1.54 ± \footnotesize{0.01} & 1.51 ± \footnotesize{0.01} & \textbf{0.9} ± \footnotesize{0.01} \\
EasyUQ & 6.5 ± \footnotesize{0.08} & 182.3 ± \footnotesize{1.47} & 310.1 & 255.9 & 1.22 ± \footnotesize{0.01} & 2.25 ± \footnotesize{0.02} & 6.2 ± \footnotesize{0.02} \\
Analogues & 7.5 ± \footnotesize{0.12} & 241.2 ± \footnotesize{2.7} & 410.6 & 338.1 & 1.68 ± \footnotesize{0.01} & 1.63 ± \footnotesize{0.01} & 1.3 ± \footnotesize{0.01} \\
GAN & 9.2 ± \footnotesize{0.09} & 223.4 ± \footnotesize{1.44} & 376.1 & 305.4 & 1.48 ± \footnotesize{0.01} & 1.41 ± \footnotesize{0.01} & 2.4 ± \footnotesize{0.02} \\
CorrDiff & 6.5 ± \footnotesize{0.1} & 212.5 ± \footnotesize{2.21} & 396.6 & 36

/tmp/ipykernel_1642209/2588688811.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel[se_col] = df_sel[se_col].round(2)
/tmp/ipykernel_1642209/2588688811.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel[col] = df_sel[col].apply(lambda x: f'\\textbf{{{x:.1f}}}' if x == df_sel[col].iloc[min_idx] else (f'\\underline{{{x:.1f}}}' if x == df_sel[col].iloc[second_min_idx] else f'{x:.1f}'))
/tmp/ipykernel_1642209/2588688811.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

## Extrapolation

In [4]:
import pandas as pd
import numpy as np 

In [5]:
method_list

['nn_det_per_variable',
 'idr',
 'analogues',
 'gan',
 'corrdiff_fixed-seeds',
 'maybritt_nicolai_zeros-constant_dec-1e-3_onehot',
 'maybritt_nicolai_zeros-constant_dec-1e-3_onehot_temporal']

In [6]:
method_name_dict_extra = {k: v + "-extra" for k, v in method_name_dict.items()}
latex_tables = []
    
for var in ['tas', 'pr', 'sfcWind', 'rsds']:
    if folder_name_maybritt is not None:
        df = pd.read_csv(f"output_evals/maybritt/extrapolation/{folder_name_maybritt}/{var}/concatenated_results_2090-2099_multiple-methods_2378111217.csv")
    if folder_name_benchmarks is not None:
        df_bench = pd.read_csv(f"output_evals/benchmarks/extrapolation/{folder_name_benchmarks}/{var}/concatenated_results_2090-2099_maybritt_3step_dense-conv_v3_2378111217.csv")
    
    if folder_name_maybritt is not None and folder_name_benchmarks is not None:
        df = pd.concat([df, df_bench])
    elif folder_name_benchmarks is not None:
        df = df_bench
    
    df = df.set_index('method').loc[method_list].reset_index()

    col_list = []
    for name in ["MSE", "ES_pool-None_kernel-1", "Score1_pool-None_kernel-1", "Score2_pool-None_kernel-1", "CRPS_pool-None_kernel-1", "CRPS_pool-max_kernel-10", "LSD-daily"]:
        col_list.append("mean*" + name)
        col_list.append("se*" + name)

    df_sel = df[["method"] + col_list] 
    for col in df_sel.columns:
        if col.startswith('mean*'):
            se_col = col.replace('mean*', 'se*')
            #df_sel[col] = df_sel[col].round(1)
            df_sel[se_col] = df_sel[se_col].round(2)
            
            if not (col in ["mean*Score1_pool-None_kernel-1", "mean*Score2_pool-None_kernel-1"]):
                min_idx = df_sel[col].idxmin()
                second_min_idx = df_sel[col].nsmallest(2).idxmax()
                if col in ["mean*CRPS_pool-None_kernel-1", "mean*CRPS_pool-max_kernel-10"]:
                    df_sel[col] = df_sel[col].apply(lambda x: f'\\textbf{{{x:.2f}}}' if x == df_sel[col].iloc[min_idx] else (f'\\underline{{{x:.2f}}}' if x == df_sel[col].iloc[second_min_idx] else f'{x:.2f}'))
                else:
                    # df_sel[col] = df_sel[col].apply(lambda x: f'\\textbf{{{x:.1f}}}' if x == df_sel[col].iloc[min_idx] else f'{x:.1f}')
                    df_sel[col] = df_sel[col].apply(lambda x: f'\\textbf{{{x:.1f}}}' if x == df_sel[col].iloc[min_idx] else (f'\\underline{{{x:.1f}}}' if x == df_sel[col].iloc[second_min_idx] else f'{x:.1f}'))
            else:
                df_sel[col] = df_sel[col].apply(lambda x: f'{x:.1f}')
                
            
            if not (col in ["mean*Score1_pool-None_kernel-1", "mean*Score2_pool-None_kernel-1"]):
                df_sel[col] = df_sel[col].astype(str) + ' ± ' + "\\footnotesize{" + df_sel[se_col].astype(str) + "}"
            df_sel.rename(columns={col: col.replace('mean*', '')}, inplace=True)
            df_sel = df_sel.drop(columns=[se_col])

        #elif col == "LSD":
        #    min_idx = df_sel[col].idxmin()
        #    second_min_idx = df_sel[col].nsmallest(2).idxmax()
        #    # df_sel[col] = df_sel[col].apply(lambda x: f'\\textbf{{{x:.3f}}}' if x == df_sel[col].iloc[min_idx] else f'{x:.3f}')
        #    df_sel[col] = df_sel[col].apply(lambda x: f'\\textbf{{{x:.3f}}}' if x == df_sel[col].iloc[min_idx] else (f'\\underline{{{x:.3f}}}' if x == df_sel[col].iloc[second_min_idx] else f'{x:.3f}'))

    df_sel.rename(columns={"MSE": "MSE $\\downarrow$",
                            "ES_pool-None_kernel-1": "ES $\\downarrow$", 
                           "Score1_pool-None_kernel-1": "ES$_\\text{pred}$", "Score2_pool-None_kernel-1": "ES$_\\text{var}$", 
                           "CRPS_pool-None_kernel-1": "CRPS $\\downarrow$", "CRPS_pool-max_kernel-10": "CRPS-mp $\\downarrow$",
                           "LSD-daily": "RALSD $\\downarrow$"}, inplace=True)

    #df_sel = pd.concat([df_sel, df[["CRPS-pixelwise", "CRPS-max-pool-10x10"]]], axis=1) # TO DO after standard errors
    #df_sel.rename(columns={"CRPS-pixelwise": "CRPS-pw", "CRPS-max-pool-10x10": "CRPS-max-pool-10"}, inplace=True)

    # Print the LaTeX table
    # Remove trailing zeros
    # df_sel[df_sel.select_dtypes(include=[np.number]).columns] = df_sel.select_dtypes(include=[np.number]).applymap(lambda x: '{:.1f}'.format(x).rstrip('0').rstrip('.') if pd.notnull(x) else '')
        

    df_sel['method'] = df_sel['method'].replace(method_name_dict_extra)
    # Convert the DataFrame to a LaTeX table
    latex_table = df_sel.to_latex(index=False, na_rep='')
    
    latex_tables.append(latex_table)

# Concatenate LaTeX tables with \multicolumn and \midrule between variables
final_latex_table = ""
final_latex_table += "\\begin{tabular}{llllllll}\n"
for var, table in zip(['tas', 'pr', 'sfcWind', 'rsds'], latex_tables):
    final_latex_table += f"\\midrule\n\\multicolumn{{7}}{{c}}{{{var}}} \\\\\n\\midrule\n"
    final_latex_table += table.split("\\toprule")[1].split("\\bottomrule")[0].strip() + "\n"
final_latex_table += "\\end{tabular}"
print(final_latex_table)
# Save the final LaTeX table to a .tex file
with open(f'tables/table_energy_scores_crps_lsd_{version_name}_extrapolation.tex', 'w') as f:
        f.write(final_latex_table)

/tmp/ipykernel_1642209/3716671434.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel[se_col] = df_sel[se_col].round(2)
/tmp/ipykernel_1642209/3716671434.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel[col] = df_sel[col].apply(lambda x: f'\\textbf{{{x:.1f}}}' if x == df_sel[col].iloc[min_idx] else (f'\\underline{{{x:.1f}}}' if x == df_sel[col].iloc[second_min_idx] else f'{x:.1f}'))
/tmp/ipykernel_1642209/3716671434.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

\begin{tabular}{llllllll}
\midrule
\multicolumn{7}{c}{tas} \\
\midrule
method & MSE $\downarrow$ & ES $\downarrow$ & ES$_\text{pred}$ & ES$_\text{var}$ & CRPS $\downarrow$ & CRPS-mp $\downarrow$ & RALSD $\downarrow$ \\
\midrule
NN-det-extra & \textbf{4.4} ± \footnotesize{0.08} & 240.1 ± \footnotesize{1.9} & 240.0 & 0.0 & 1.51 ± \footnotesize{0.01} & 1.51 ± \footnotesize{0.01} & \textbf{0.8} ± \footnotesize{0.01} \\
EasyUQ-extra & 6.3 ± \footnotesize{0.08} & 179.7 ± \footnotesize{1.5} & 305.4 & 251.3 & 1.20 ± \footnotesize{0.01} & 2.14 ± \footnotesize{0.02} & 6.1 ± \footnotesize{0.01} \\
Analogues-extra & 8.0 ± \footnotesize{0.13} & 248.8 ± \footnotesize{2.7} & 415.5 & 332.8 & 1.74 ± \footnotesize{0.01} & 1.73 ± \footnotesize{0.01} & 1.2 ± \footnotesize{0.01} \\
GAN-extra & 9.3 ± \footnotesize{0.1} & 223.4 ± \footnotesize{1.42} & 377.4 & 308.1 & 1.48 ± \footnotesize{0.01} & 1.45 ± \footnotesize{0.01} & 2.2 ± \footnotesize{0.01} \\
CorrDiff-extra & 6.8 ± \footnotesize{0.11} & 219.5 ± \fo

/tmp/ipykernel_1642209/3716671434.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel[se_col] = df_sel[se_col].round(2)
/tmp/ipykernel_1642209/3716671434.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel[col] = df_sel[col].apply(lambda x: f'\\textbf{{{x:.1f}}}' if x == df_sel[col].iloc[min_idx] else (f'\\underline{{{x:.1f}}}' if x == df_sel[col].iloc[second_min_idx] else f'{x:.1f}'))
/tmp/ipykernel_1642209/3716671434.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [7]:
df

,method,mean*MSE,se*MSE,mean*ES_pool-None_kernel-1,se*ES_pool-None_kernel-1,mean*Score1_pool-None_kernel-1,se*Score1_pool-None_kernel-1,mean*Score2_pool-None_kernel-1,se*Score2_pool-None_kernel-1,mean*CRPS_pool-None_kernel-1,...,ACF_lag1_abs_error_pool-average_kernel-128,ACF_lag1_diff_pool-average_kernel-128,mean*LSD-daily,se*LSD-daily,mean*LSD-avg,se*LSD-avg,q-error_months-12-1-2_q-0.05,q-error_months-12-1-2_q-0.95,q-error_months-6-7-8_q-0.05,q-error_months-6-7-8_q-0.95
0,nn_det_per_variable,1914.898132,33.336583,4888.127258,40.518630,4883.912445,43.407486,0.000000,0.000000,29.893853,...,0.000660,0.000311,3.489783,0.031056,3.579875,0.035230,6.362183,11.181025,41.468463,14.435953
1,idr,3217.844818,40.207954,3520.514866,29.428281,6613.191364,44.792612,6188.439780,39.802140,23.595247,...,0.000738,0.000259,10.343008,0.024346,10.080244,0.028599,1.094840,7.777652,8.952737,2.963572
2,analogues,2560.159998,36.129943,4277.073683,44.873217,7221.143006,55.879441,5901.286581,72.506518,28.185447,...,0.032537,-0.032537,1.844003,0.016246,0.124289,0.020374,1.005758,3.342556,9.103048,6.304642
3,gan,2906.478729,40.602973,3566.452667,31.144037,6197.648193,48.015616,5265.807617,37.086765,23.429309,...,0.006592,-0.006592,3.670139,0.032745,3.792233,0.038477,8.099106,6.600666,29.179198,12.983480
4,corrdiff_fixed-seeds,3466.694458,54.056074,3100.584930,28.902542,6243.180786,54.091783,6285.301392,53.428366,21.854483,...,0.006472,0.006472,1.543247,0.013456,0.786380,0.018012,2.885879,6.482854,27.235872,10.141816
5,maybritt_nicolai_zeros-constant_dec-1e-3_onehot,1936.009800,30.343927,3371.055200,31.240469,6930.054000,45.024307,7118.093000,49.743225,23.289181,...,0.024114,-0.024114,2.913243,0.023869,1.528334,0.035128,2.379827,12.566593,12.518537,5.908495
6,maybritt_nicolai_zeros-constant_dec-1e-3_oneho...,1989.132100,29.101753,3392.277800,30.766450,6995.557000,53.609990,7211.397000,52.440940,23.410392,...,0.004106,-0.004106,2.899339,0.024214,1.528545,0.035406,2.035773,12.228483,14.014818,6.672264


### Extremes univarate

#### Merge quantiles and calibration of quantiles

In [8]:
# Read the CSV file
latex_tables = []
dfs = []

for var in ['tas', 'pr', 'sfcWind', 'rsds']:
    if folder_name_maybritt is not None:
        df = pd.read_csv(f"output_evals/maybritt/interpolation/{folder_name_maybritt}/{var}/concatenated_results_2030-2039_multiple-methods_2378111217.csv")
    if folder_name_benchmarks is not None:
        df_bench = pd.read_csv(f"output_evals/benchmarks/interpolation/{folder_name_benchmarks}/{var}/concatenated_results_2030-2039_maybritt_3step_dense-conv_v3_2378111217.csv")
    
    if folder_name_maybritt is not None and folder_name_benchmarks is not None:
        df = pd.concat([df, df_bench])
    elif folder_name_benchmarks is not None:
        df = df_bench
        
    df = df.set_index('method').loc[method_list].reset_index()

    col_list = []   

    col_list.append("q-error_months-12-1-2_q-0.05")
    col_list.append("q-error_months-6-7-8_q-0.95")
    col_list.append("loc_avg_mcb_first_bin")
    col_list.append("loc_avg_mcb_last_bin")
    
    df_sel = df[["method"] + col_list] 
        
    for col in df_sel.select_dtypes(include=[np.number]).columns:
        # Find the index of the minimum and second minimum value in the column
        min_idx = df_sel[col].idxmin()
        second_min_idx = df_sel[col].nsmallest(2).idxmax()
        
        # Apply bold formatting to the lowest value and underline to the second lowest value
        df_sel[col] = df_sel[col].apply(lambda x: f'\\textbf{{{x:.3f}}}' if x == df_sel[col].iloc[min_idx] else (f'\\underline{{{x:.3f}}}' if x == df_sel[col].iloc[second_min_idx] else f'{x:.3f}'))

    df_sel['method'] = df_sel['method'].replace(method_name_dict)
    # rename columns and remove QL none
    df_sel.rename(columns={"loc_avg_mcb_first_bin": "MCB $Q^{0.1}$ $\\downarrow$", "loc_avg_mcb_last_bin": "MCB $Q^{0.9}$ $\\downarrow$",
                           "q-error_months-12-1-2_q-0.05": "AE $Q^{0.05}_{\\text{Wi}}\\downarrow$",
                           "q-error_months-6-7-8_q-0.95": "AE $Q^{0.95}_{\\text{Su}}\\downarrow$",}, inplace=True)
            # Convert the DataFrame to a LaTeX table
    latex_table = df_sel.to_latex(index=False, na_rep='', multirow=True, multicolumn_format='c')

    # Insert lines after the first 4 and 7 columns
    #latex_table = latex_table.replace(' & QL None & QL mean & QL max', ' & QL None & QL mean & QL max \\\\\n\\cmidrule(lr){2-4}\\cmidrule(lr){5-7}\\cmidrule(lr){8-10}')

    latex_tables.append(latex_table)
    dfs.append(df_sel)

# Concatenate LaTeX tables with \multicolumn and \midrule between variables
final_latex_table = ""
final_latex_table += "\\begin{tabular}{lllll|llll}\n"
# for var, table in zip(['tas', 'pr', 'sfcWind', 'rsds'], latex_tables):

final_latex_table += f"\\midrule\nMethod&\\multicolumn{{4}}{{c}}{{tas}}&\\multicolumn{{3}}{{|c}}{{pr}}\\\\\n\\midrule\n"
df_tas_pr = pd.concat([dfs[0], dfs[1].iloc[:, 1:]], axis=1)
latex_str = df_tas_pr.to_latex(index=False, escape=False)
latex_str = "\n".join(latex_str.splitlines()[2:-2])
final_latex_table += latex_str + "\n"
final_latex_table += f"\\midrule\nMethod&\\multicolumn{{4}}{{c}}{{sfcWind}}&\\multicolumn{{3}}{{|c}}{{rsds}}\\\\\n\\midrule\n"
df_sfcWind_rsds = pd.concat([dfs[2], dfs[3].iloc[:, 1:]], axis=1)
latex_str = df_sfcWind_rsds.to_latex(index=False, escape=False)
latex_str = "\n".join(latex_str.splitlines()[2:-1])
final_latex_table += latex_str + "\n"
final_latex_table += "\\end{tabular}\n"
print(final_latex_table)

with open(f'tables/table_quantiles_with_mcb_{version_name}.tex', 'w') as f:
    f.write(final_latex_table)

\begin{tabular}{lllll|llll}
\midrule
Method&\multicolumn{4}{c}{tas}&\multicolumn{3}{|c}{pr}\\
\midrule
method & AE $Q^{0.05}_{\text{Wi}}\downarrow$ & AE $Q^{0.95}_{\text{Su}}\downarrow$ & MCB $Q^{0.1}$ $\downarrow$ & MCB $Q^{0.9}$ $\downarrow$ & AE $Q^{0.05}_{\text{Wi}}\downarrow$ & AE $Q^{0.95}_{\text{Su}}\downarrow$ & MCB $Q^{0.1}$ $\downarrow$ & MCB $Q^{0.9}$ $\downarrow$ \\
\midrule
NN-det & 1.021 & 0.948 & nan & nan & 0.007 & 6.960 & nan & nan \\
EasyUQ & \underline{0.539} & \underline{0.479} & 0.060 & 0.064 & \underline{0.000} & \underline{2.459} & \textbf{0.028} & 0.039 \\
Analogues & 0.847 & 0.485 & 0.103 & 0.054 & \textbf{0.000} & \textbf{1.449} & 0.068 & 0.085 \\
GAN & 1.606 & 1.545 & 0.097 & 0.063 & 0.012 & 4.395 & 0.274 & 0.038 \\
CorrDiff & 1.069 & 1.086 & 0.056 & 0.050 & 0.044 & 5.327 & 0.088 & 0.047 \\
EnScale & \textbf{0.488} & \textbf{0.478} & \textbf{0.021} & \underline{0.021} & 0.000 & 2.813 & \underline{0.045} & \textbf{0.018} \\
EnScale-t & 0.673 & 0.817 & \underli

/tmp/ipykernel_1642209/3999571305.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel[col] = df_sel[col].apply(lambda x: f'\\textbf{{{x:.3f}}}' if x == df_sel[col].iloc[min_idx] else (f'\\underline{{{x:.3f}}}' if x == df_sel[col].iloc[second_min_idx] else f'{x:.3f}'))
/tmp/ipykernel_1642209/3999571305.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel[col] = df_sel[col].apply(lambda x: f'\\textbf{{{x:.3f}}}' if x == df_sel[col].iloc[min_idx] else (f'\\underline{{{x:.3f}}}' if x == df_sel[co

## Correlations and RHs for differences

In [9]:

# NEW
if folder_name_maybritt is not None:
    df1 = pd.read_csv(f"output_evals/maybritt/interpolation/{folder_name_maybritt}/multivariate_concatenated_results_2030-2039_multiple-methods_1315.csv")
if folder_name_benchmarks is not None:
    df_bench1 = pd.read_csv(f"output_evals/benchmarks/interpolation/{folder_name_benchmarks}/multivariate_concatenated_results_2030-2039_maybritt_3step_dense-conv_v3_1315.csv")

if folder_name_maybritt is not None and folder_name_benchmarks is not None:
    df_multi = pd.concat([df1, df_bench1])
elif folder_name_benchmarks is not None:
    df_multi = df_bench1
df = df_multi
print(df.columns)
df['var_combined'] = df['var1'] + '-' + df['var2']
# RESCALE THE MCB VALUES because num_locs was just 1000
df['mcb_loc_average'] = df['mcb_loc_average'] * 128*128/1000
df['rh_mean_loc_average'] = df['rh_mean_loc_average'] * 128*128/1000
df['rh_var_loc_average'] = df['rh_var_loc_average'] * 128*128/1000

# calibration for NN-det makes no sense
df.loc[df['method'] == 'nn_det_per_variable', 'mcb_loc_average'] = np.nan
df_pivot = df.pivot_table(index='method', columns='var_combined', values=['PW corrs pixelwise', 'mcb_loc_average'])
df_pivot.reset_index(inplace=True)
df_sel = df_pivot.copy()
#df_sel = df_pivot.drop(index=[1, 6, 8])
df_sel.reset_index(drop=True, inplace=True)
df_sel = df_sel.set_index('method').loc[method_list].reset_index()
df_sel['method'] = df_sel['method'].replace(method_name_dict)
df_sel.rename(columns={
    col[0]: "Correlations" if col[0] == "PW corrs pixelwise"
         else "Miscalibration" if col[0] == "mcb_loc_average"
         else "Method" if col[0] == "method"
         else col[0]
    for col in df_sel.columns
}, inplace=True)

# Add bold for lowest number
for col in df_sel.select_dtypes(include=[np.number]).columns:
    # Find the index of the minimum and second minimum value in the column
    min_idx = df_sel[col].idxmin()
    second_min_idx = df_sel[col].nsmallest(2).idxmax()
    
    # Apply bold formatting to the lowest value and underline to the second lowest value
    df_sel[col] = df_sel[col].apply(lambda x: f'\\textbf{{{x:.3f}}}' if x == df_sel[col].iloc[min_idx] else (f'\\underline{{{x:.3f}}}' if x == df_sel[col].iloc[second_min_idx] else f'{x:.3f}'))


final_latex_table = df_sel.to_latex(index=False)

with open(f'tables/table_multivariate_updated_{version_name}.tex', 'w') as f:
    f.write(final_latex_table)
    
print(final_latex_table)

Index(['method', 'PW corrs pixelwise', 'PW corrs max-pool 10x10',
       'PW corrs avg-pool 10x10', 'var1', 'var2', 'mcb_loc_average',
       'rh_mean_loc_average', 'rh_var_loc_average'],
      dtype='object')
\begin{tabular}{lllllll}
\toprule
Method & \multicolumn{3}{r}{Correlations} & \multicolumn{3}{r}{Miscalibration} \\
 & pr-sfcWind & sfcWind-rsds & tas-pr & pr-sfcWind & sfcWind-rsds & tas-pr \\
\midrule
NN-det & 0.125 & 0.062 & 0.060 & nan & nan & nan \\
EasyUQ & 0.166 & 0.217 & 0.055 & 0.563 & 0.790 & 0.389 \\
Analogues & \textbf{0.026} & \underline{0.029} & \textbf{0.025} & 0.355 & 0.331 & 0.325 \\
GAN & 0.063 & 0.056 & 0.042 & 0.098 & 0.092 & 0.123 \\
CorrDiff & 0.126 & 0.094 & 0.117 & 0.195 & 0.192 & 0.213 \\
EnScale & 0.044 & \textbf{0.029} & \underline{0.031} & \textbf{0.055} & \underline{0.054} & \underline{0.067} \\
EnScale-t & \underline{0.042} & 0.031 & 0.033 & \underline{0.056} & \textbf{0.054} & \textbf{0.067} \\
\bottomrule
\end{tabular}



## ACF

Signed error and abs error separately

In [10]:

dfs_error = []
dfs_abs_error = []

for var in ['tas', 'pr', 'sfcWind', 'rsds']:
    if folder_name_maybritt is not None:
        df = pd.read_csv(f"output_evals/maybritt/interpolation/{folder_name_maybritt}/{var}/concatenated_results_2030-2039_multiple-methods_2378111217.csv")
    if folder_name_benchmarks is not None:
        df_bench = pd.read_csv(f"output_evals/benchmarks/interpolation/{folder_name_benchmarks}/{var}/concatenated_results_2030-2039_maybritt_3step_dense-conv_v3_2378111217.csv")
    
    if folder_name_maybritt is not None and folder_name_benchmarks is not None:
        df = pd.concat([df, df_bench])
    elif folder_name_benchmarks is not None:
        df = df_bench
        
    df = df.set_index('method').loc[method_list].reset_index()

    # Extract relevant columns
    df_error = df[["method", "ACF_lag1_diff_pool-None_kernel-1"]].copy()
    df_abs_error = df[["method", "ACF_lag1_abs_error_pool-None_kernel-1"]].copy()
    
    # Rename columns
    df_error.rename(columns={"ACF_lag1_diff_pool-None_kernel-1": f"{var}", "method": "Method"}, inplace=True)
    df_abs_error.rename(columns={"ACF_lag1_abs_error_pool-None_kernel-1": f"{var}",  "method": "Method"}, inplace=True)
    
    # Format floats
    for subdf in [df_error, df_abs_error]:
        subdf[subdf.select_dtypes(include=[np.number]).columns] = subdf.select_dtypes(include=[np.number]).applymap(
            lambda x: '{:.2f}'.format(x).rstrip('0').rstrip('.') if pd.notnull(x) else ''
        )

        subdf['Method'] = subdf['Method'].replace(method_name_dict)

    dfs_error.append(df_error)
    dfs_abs_error.append(df_abs_error)

# Merge all by method
df_error_all = dfs_error[0]
df_abs_error_all = dfs_abs_error[0]
for i in range(1, len(dfs_error)):
    df_error_all = pd.merge(df_error_all, dfs_error[i], on='Method')
    df_abs_error_all = pd.merge(df_abs_error_all, dfs_abs_error[i], on='Method')

# Reorder columns: method first
cols_ordered_error = ['Method'] + [col for col in df_error_all.columns if col != 'Method']
cols_ordered_abs_error = ['Method'] + [col for col in df_abs_error_all.columns if col != 'Method']
df_error_all = df_error_all[cols_ordered_error]
df_abs_error_all = df_abs_error_all[cols_ordered_abs_error]

# Save LaTeX tables
latex_error = df_error_all.to_latex(index=False)
latex_abs_error = df_abs_error_all.to_latex(index=False)

with open(f'tables/table_acfs_signed_error_{version_name}.tex', 'w') as f:
    f.write(latex_error)

with open(f'tables/table_acfs_abs_error_{version_name}.tex', 'w') as f:
    f.write(latex_abs_error)

/tmp/ipykernel_1642209/3104341603.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  subdf[subdf.select_dtypes(include=[np.number]).columns] = subdf.select_dtypes(include=[np.number]).applymap(
/tmp/ipykernel_1642209/3104341603.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  subdf[subdf.select_dtypes(include=[np.number]).columns] = subdf.select_dtypes(include=[np.number]).applymap(
/tmp/ipykernel_1642209/3104341603.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  subdf[subdf.select_dtypes(include=[np.number]).columns] = subdf.select_dtypes(include=[np.number]).applymap(
/tmp/ipykernel_1642209/3104341603.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  subdf[subdf.select_dtypes(include=[np.number]).columns] = subdf.select_dtypes(include=[np.number]).applymap(


# Variability

## Coarse vs. super

In [11]:
import torch
from utils import get_rcm_gcm_combinations

In [12]:
# idx0_inter_gen-coarse.pt
gcm_list, rcm_list, gcm_dict, rcm_dict = get_rcm_gcm_combinations("/r/scratch/groups/nm/downscaling/cordex-ALPS-allyear")
run_index = 0

run_ids = [maybritt_runid]

for k in range(8):
    gen_data = {}
    name_res = "inter"
    variables = ["tas", "pr", "sfcWind", "rsds"]
    for run_id in run_ids:
        gen_data[run_id] = {}
        dir_m = f"/r/scratch/groups/nm/downscaling/samples_multivariate/" + run_id
        # idx0_inter
        data = torch.load(f"{dir_m}/idx{k}_{name_res}_gen-super-from-coarse.pt")
        var_order = ["tas", "pr", "sfcWind", "rsds"]
        for var in variables:
            if run_id.endswith("test-rsds") or run_id.endswith("test-rsds2"):
                data_var = data[:, 0, :, :]
            else:
                data_var = data[:, var_order.index(var), :, :]
            gen_data[run_id][var] = data_var.view(-1, 128, 128, 9*9).numpy()
            
    # Calculate the average of the fields
    avg_cond_var_list = []
    avg_var_cond_list = []

    for var in variables:
        var_list = [np.var(gen_data[run_ids[0]][var][...,(k*9):((k+1)*9)], axis=-1) for k in range(9)]
        avg_cond_var = np.stack(var_list, axis=-1).mean(axis=-1).mean()
        avg_cond_var_list.append(avg_cond_var)

        mean_list = [np.mean(gen_data[run_ids[0]][var][...,(k*9):((k+1)*9)], axis=-1) for k in range(9)]
        avg_var_cond = np.stack(mean_list, axis=-1).var(axis=-1).mean()
        avg_var_cond_list.append(avg_var_cond)

    # Create a DataFrame to store the results
    df_avg_fields = pd.DataFrame({
        'Variable': variables,
        'Conditional Var. Y|Z': avg_cond_var_list,
        'Var. of E[Y|Z]': avg_var_cond_list
    })

    df_avg_fields["gcm"] = gcm_list[k]
    df_avg_fields["rcm"] = rcm_list[k]
    
    if k == 0:
        df_avg_fields_all = df_avg_fields
    else:
        df_avg_fields_all = pd.concat([df_avg_fields_all, df_avg_fields], axis=0)

KeyboardInterrupt: 

In [ ]:
# Reorder the DataFrame based on the specified order
order = ["tas", "pr", "sfcWind", "rsds"]
df_reordered = df_avg_fields_all.set_index('Variable').loc[order].reset_index()
# Create a LaTeX table from the reordered DataFrame
table_reordered = df_reordered[["Variable", "Conditional Var. Y|Z", "Var. of E[Y|Z]"]]
table_reordered["Ratio"] = table_reordered["Var. of E[Y|Z]"] / table_reordered["Conditional Var. Y|Z"] 

# Group by 'Variable' and calculate the mean for each group
table_grouped = table_reordered.groupby('Variable').mean().reset_index()
print(table_grouped)

  Variable  Conditional Var. Y|Z  Var. of E[Y|Z]      Ratio
0       pr              8.912724       17.150085   1.957404
1     rsds            287.631836     1387.638062   4.809596
2  sfcWind              0.309537        1.856706   6.048446
3      tas              0.236938        3.973680  16.755035


/tmp/ipykernel_230051/3895613968.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_reordered["Ratio"] = table_reordered["Var. of E[Y|Z]"] / table_reordered["Conditional Var. Y|Z"]


In [ ]:
table_grouped.rename(columns={"Var. of E[Y|Z]": "Coarse-scale variance", "Conditional Var. Y|Z": "Local-scale variance", "Ratio": "Ratio (coarse/local)"}, inplace=True)
latex_table = table_grouped.iloc[[3,0,2,1], :][["Variable", "Coarse-scale variance", "Local-scale variance", "Ratio (coarse/local)"]].to_latex(index=False, float_format="%.2f")
print(latex_table)
with open(f'tables/table_decomposition_variance_{version_name}.tex', 'w') as f:
    f.write(latex_table)

\begin{tabular}{lrrr}
\toprule
Variable & Coarse-scale variance & Local-scale variance & Ratio (coarse/local) \\
\midrule
tas & 3.97 & 0.24 & 16.76 \\
pr & 17.15 & 8.91 & 1.96 \\
sfcWind & 1.86 & 0.31 & 6.05 \\
rsds & 1387.64 & 287.63 & 4.81 \\
\bottomrule
\end{tabular}



## Counterfactuals

### Counterfactuals

In [ ]:
from utils import load_data_counterfact

In [ ]:
rcm = "ALADIN63"
gcm = "CNRM-CM5"
variables = ["tas", "pr", "sfcWind", "rsds"]
# true_data, gen_data, true_data_counterfact = load_data_counterfact(gcm, rcm, variables, mode = "interpolation", run_id = "maybritt_3step_dense_v1", rcm_counterfact = "CCLM4-8-17")
run_id = maybritt_runid
true_data, gen_data, true_data_counterfact = load_data_counterfact(gcm, rcm, variables, mode = "interpolation", run_id = run_id, rcm_counterfact = "CCLM4-8-17")

In [ ]:
import pandas as pd
import numpy as np

# Initialize a list to store the results
results1 = []

for idx, var in enumerate(variables):
    field1 = np.var(gen_data["maybritt_3step_base"][var], axis=-1).mean(axis=0)
    field2 = np.var(gen_data["maybritt_3step_counterfact"][var], axis=-1).mean(axis=0)
    mean_group1 = np.mean(gen_data["maybritt_3step_base"][var], axis=-1)
    mean_group2 = np.mean(gen_data["maybritt_3step_counterfact"][var], axis=-1)
    mean_all = np.mean(np.concatenate([gen_data["maybritt_3step_base"][var], gen_data["maybritt_3step_counterfact"][var]], axis=-1), axis=-1)
    field3 = ((mean_group1 - mean_all)**2 + (mean_group2 - mean_all)**2).mean(axis=0) * 0.5

    # Calculate the spatial averages
    avg_field1 = field1.mean()
    avg_field2 = field2.mean()
    avg_field3 = field3.mean()

    # Append the results to the list
    results1.append([var, avg_field1, avg_field2, avg_field3])

In [ ]:
df_results1 = pd.DataFrame(results1, columns=['Variable', 'Within-group Variance Y|X (RCM 1)', 'Within-group Variance Y|X (RCM 2)', 'Between-group Variance Y|X'])
df_results1

,Variable,Within-group Variance Y|X (RCM 1),Within-group Variance Y|X (RCM 2),Between-group Variance Y|X
0,tas,4.589704,4.247896,0.420785
1,pr,31.270874,24.072926,1.248032
2,sfcWind,2.093510,2.058642,0.138691
3,rsds,1853.470093,1888.615234,294.078705


In [ ]:
df_results1.columns = ['Variable', 'Within-group Var. (RCM1)', 'Within-group Var. (RCM 2)', 'Model difference']
df_results1["Variance of RCM"] = (df_results1["Within-group Var. (RCM1)"] + df_results1["Within-group Var. (RCM 2)"]) / 2
df_results1["Ratio (Variance / model diff.)"] = df_results1["Variance of RCM"] / df_results1["Model difference"] 
# print(df_results1)
df_results1 = df_results1[['Variable', 'Variance of RCM', 'Model difference', 'Ratio (Variance / model diff.)']]
latex_table = df_results1.to_latex(index=False, float_format="%.2f")
print(latex_table)

with open(f'tables/table_decomposition_var-vs-modeldiff_{version_name}.tex', 'w') as f:
    f.write(latex_table)

\begin{tabular}{lrrr}
\toprule
Variable & Variance of RCM & Model difference & Ratio (Variance / model diff.) \\
\midrule
tas & 4.42 & 0.42 & 10.50 \\
pr & 27.67 & 1.25 & 22.17 \\
sfcWind & 2.08 & 0.14 & 14.97 \\
rsds & 1871.04 & 294.08 & 6.36 \\
\bottomrule
\end{tabular}



# Appendix: PSDs

In [13]:
# Read the CSV file
latex_tables = []
dfs = []
for var in ['tas', 'pr', 'sfcWind', 'rsds']:
    if folder_name_maybritt is not None:
        df = pd.read_csv(f"output_evals/maybritt/interpolation/{folder_name_maybritt}/{var}/concatenated_results_2030-2039_multiple-methods_2378111217.csv")
    if folder_name_benchmarks is not None:
        df_bench = pd.read_csv(f"output_evals/benchmarks/interpolation/{folder_name_benchmarks}/{var}/concatenated_results_2030-2039_maybritt_3step_dense-conv_v3_2378111217.csv")
    
    if folder_name_maybritt is not None and folder_name_benchmarks is not None:
        df = pd.concat([df, df_bench])
    elif folder_name_benchmarks is not None:
        df = df_bench
        
    df = df.set_index('method').loc[method_list].reset_index()

    col_list = [] 
    
    for name in ["LSD-avg"]:
        col_list.append("mean*" + name)
        col_list.append("se*" + name)

    df_sel = df[["method"] + col_list] 

    for col in df_sel.columns:
        if col.startswith('mean*'):
            se_col = col.replace('mean*', 'se*')
            df_sel[col] = df_sel[col].round(3)
            df_sel[se_col] = df_sel[se_col].round(2)
            # df_sel[col] = df_sel[col].astype(str) + ' ± ' + df_sel[se_col].astype(str)
            df_sel.rename(columns={col: col.replace('mean*', '')}, inplace=True)
            df_sel = df_sel.drop(columns=[se_col])
        
    for col in df_sel.select_dtypes(include=[np.number]).columns:
        # Find the index of the minimum and second minimum value in the column
        min_idx = df_sel[col].idxmin()
        second_min_idx = df_sel[col].nsmallest(2).idxmax()
        
        # Apply bold formatting to the lowest value and underline to the second lowest value
        df_sel[col] = df_sel[col].apply(lambda x: f'\\textbf{{{x:.3f}}}' if x == df_sel[col].iloc[min_idx] else (f'\\underline{{{x:.3f}}}' if x == df_sel[col].iloc[second_min_idx] else f'{x:.3f}'))

    df_sel['method'] = df_sel['method'].replace(method_name_dict)
    #rename columns to include variable
    df_sel.rename(columns={"LSD-avg": f"{var}"}, inplace=True)
    dfs.append(df_sel)
    
# Concatenate all dataframes
df_all = dfs[0]
for i in range(1, len(dfs)):
    df_all = pd.merge(df_all, dfs[i], on='method')

# Reorder columns: method first
cols_ordered = ['method'] + [col for col in df_all.columns if col != 'method']
df_all = df_all[cols_ordered]

# Convert the DataFrame to a LaTeX table
latex_table = df_all.to_latex(index=False, na_rep='', multirow=True, multicolumn_format='c')
print(latex_table)

with open(f'tables/table_psd_{version_name}.tex', 'w') as f:
    f.write(latex_table)

\begin{tabular}{lllll}
\toprule
method & tas & pr & sfcWind & rsds \\
\midrule
NN-det & 0.466 & 8.598 & 1.392 & 4.050 \\
EasyUQ & 5.958 & 7.610 & 8.501 & 9.966 \\
Analogues & \textbf{0.150} & \textbf{0.344} & \textbf{0.153} & \textbf{0.170} \\
GAN & 1.999 & 6.890 & 3.183 & 4.066 \\
CorrDiff & \underline{0.394} & \underline{0.951} & \underline{0.432} & \underline{0.752} \\
EnScale & 0.579 & 1.456 & 0.707 & 1.293 \\
EnScale-t & 0.589 & 1.429 & 0.654 & 1.274 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_1642209/177562718.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel[col] = df_sel[col].round(3)
/tmp/ipykernel_1642209/177562718.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel[se_col] = df_sel[se_col].round(2)
/tmp/ipykernel_1642209/177562718.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-